In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas
!pip install scikit_learn
!pip install matplotlib
!pip install numpy
!pip install seaborn

In [3]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns

import re
import nltk
%matplotlib inline

In [30]:
def apply_height_width(dimen):
    a=""
    return dimen/10


def apply_depth(width):
    return width/10


def filterprice(price):
  return int(price.replace('₹',''))

def weight_filter(weight):
  return weight[:2]

In [27]:
df=pd.read_excel('/content/drive/MyDrive/Product_similarity/Main Output_2024-07-15_lg.xlsx')
df.columns
df.drop(columns=['Product Name'],inplace=True)
df.dropna(inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 1 to 48
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Source            23 non-null     object 
 1   Model Name        23 non-null     object 
 2   Date              23 non-null     object 
 3   Time              23 non-null     object 
 4   Original_MRP      23 non-null     object 
 5   Discounted_Price  23 non-null     object 
 6   Product Height    23 non-null     float64
 7   Product Width     23 non-null     float64
 8   Product Depth     23 non-null     float64
 9   Machine Weight    23 non-null     object 
 10  Washing Capacity  23 non-null     float64
 11  Product Feature   23 non-null     object 
dtypes: float64(4), object(8)
memory usage: 2.3+ KB


In [28]:
df['Discounted_Price']=df['Discounted_Price'].apply(filterprice)
df['Original_MRP']=df['Original_MRP'].apply(filterprice)
df.head(5)

,Source,Model Name,Date,Time,Original_MRP,Discounted_Price,Product Height,Product Width,Product Depth,Machine Weight,Washing Capacity,Product Feature
1,LG,"LG 6.5Kg Top Load Washing Machine, Smart Inver...",2024-07-15,10:42:07,24990,16491,870.0,540.0,560.0,30.5,6.5,3 Motion End of Cycle Signal Auto Restart Load...
2,LG,"LG 7Kg Top Load Washing Machine, Smart Inverte...",2024-07-15,10:42:07,30490,19990,925.0,540.0,560.0,32,7.0,3 Motion End of Cycle Signal Auto Restart Load...
3,LG,"LG 7Kg Top Load Washing Machine, Smart Inverte...",2024-07-15,10:42:07,28490,17490,910.0,540.0,560.0,31,7.0,3 Motion End of Cycle Signal Auto Restart Load...
5,LG,"LG 7Kg Front Load Washing Machine, AI Direct D...",2024-07-15,10:42:07,52990,35990,850.0,600.0,455.0,63kg,7.0,Steam ThinQ Wi-Fi Steam ThinQ Wi-Fi Delay Time...
8,LG,"LG 7.5Kg Top Load Washing Machine, Smart Inver...",2024-07-15,10:42:07,29490,20490,910.0,540.0,560.0,31,7.5,3 Motion End of Cycle Signal Auto Restart Load...


In [29]:
#applying filter on value of width height and depth
df['Product Height']= df['Product Height'].apply(apply_height_width)
df['Product Width']= df['Product Width'].apply(apply_height_width)
df['Product Depth']= df['Product Depth'].apply(apply_depth)
df['Machine Weight']=df['Machine Weight'].apply(weight_filter)
df['combinedtext']=df['Model Name']+df['Product Feature']
df.drop(columns=['Date','Time','Source','Model Name','Product Feature'],inplace=True)
df.head(5)


,Original_MRP,Discounted_Price,Product Height,Product Width,Product Depth,Machine Weight,Washing Capacity,combinedtext
1,24990,16491,87.0,54.0,56.0,30.5,6.5,"LG 6.5Kg Top Load Washing Machine, Smart Inver..."
2,30490,19990,92.5,54.0,56.0,32,7.0,"LG 7Kg Top Load Washing Machine, Smart Inverte..."
3,28490,17490,91.0,54.0,56.0,31,7.0,"LG 7Kg Top Load Washing Machine, Smart Inverte..."
5,52990,35990,85.0,60.0,45.5,63kg,7.0,"LG 7Kg Front Load Washing Machine, AI Direct D..."
8,29490,20490,91.0,54.0,56.0,31,7.5,"LG 7.5Kg Top Load Washing Machine, Smart Inver..."


In [32]:
def taking_text_with_label(df):
  indesxing=[]
  for i in range(len(df)+1):
    indesxing.append(int(i+1))
  indes=pd.DataFrame(indesxing,columns=['ProductNo'])
  df3=pd.concat([indes,df],axis=1)
  df3.dropna(inplace=True)
  return df3

dt_text=taking_text_with_label(df)
dt_text.head(5)

,ProductNo,Original_MRP,Discounted_Price,Product Height,Product Width,Product Depth,Machine Weight,Washing Capacity,combinedtext
1,2.0,24990.0,16491.0,87.0,54.0,56.0,30,6.5,"LG 6.5Kg Top Load Washing Machine, Smart Inver..."
2,3.0,30490.0,19990.0,92.5,54.0,56.0,32,7.0,"LG 7Kg Top Load Washing Machine, Smart Inverte..."
3,4.0,28490.0,17490.0,91.0,54.0,56.0,31,7.0,"LG 7Kg Top Load Washing Machine, Smart Inverte..."
5,6.0,52990.0,35990.0,85.0,60.0,45.5,63,7.0,"LG 7Kg Front Load Washing Machine, AI Direct D..."
8,9.0,29490.0,20490.0,91.0,54.0,56.0,31,7.5,"LG 7.5Kg Top Load Washing Machine, Smart Inver..."


In [34]:
df.to_csv('/content/drive/MyDrive/Product_similarity/df.csv')